找到具有最小间隔的数据点，再对该间隔最大化：
$$
\mathop{\arg\max}_{(w,b)}\{\min_n \ label·\frac{(w^Tx+b)}{||w||}\}
$$
转化后的优目标函数：
$$
\begin{align}
\max_\alpha[\sum_{i=1}^m\alpha-\frac{1}{2}\sum_{i,j=1}^{m}&label^{(i)}·label^{(j)}·\alpha_i·\alpha_j·<x^{(i)},x^{(j)}>] \\\\
st. \ &\alpha\geq0\\
&\sum_{i-1}^m\alpha_i·label^{(i)}=0
\end{align}
$$
引入松弛变量后约束条件变为：
$$
\begin{align}
st. \ &C\geq\alpha\geq0\\&\sum_{i-1}^m\alpha_i·label^{(i)}=0
\end{align}
$$


**SMO：序列最小优化（Sequential Minimal Optimization）**
将大优化问题分解为多个小优化问题  
SMO目标：求出一系列$\alpha$和$b$，然后可以容易计算出$w$  
工作原理：  
1.每次循环选择两个$\alpha$进行优化处理  
2.一旦找到一对“合适”的$\alpha$，就一个增大一个减小  

In [1]:
import numpy as np

In [2]:
# 读取数据，获得特征与标签
def loadDataSet(fileName):
    dataMat = []
    labelMat = []
    fr = open(fileName)
    for line in fr.readlines():
        lineArr = line.strip().split('\t')
        dataMat.append([float(lineArr[0]), float(lineArr[1])])
        labelMat.append(float(lineArr[2]))
    return dataMat, labelMat

# i是alpha下标，m是alpha数目
def selectJrand(i, m):
    j = i;
    while j == i:
        j = int(np.random.uniform(0, m))
    return j

# 调整大于H或小于L的alpha值
def clipAlpha(aj, H, L):
    if aj > H:
        aj = H
    if L > aj:
        aj = L
    return aj

Platt SMO支持函数

In [3]:
class optStruct:
    def __init__(self, dataMatIn, classLabels, C, toler):
        self.X = dataMatIn
        self.labelMat = classLabels
        self.C = C
        self.tol = toler
        self.m = np.shape(dataMatIn)[0]
        self.alphas = np.mat(np.zeros((self.m, 1)))
        self.b = 0
        self.eCache = np.mat(np.zeros((self.m, 2)))    # 误差缓存

# 计算E值并返回
def calcEk(oS, k):
    fXk = float(np.multiply(oS.alphas, oS.labelMat).T * (oS.X * oS.X[k, :].T)) + oS.b
    Ek = fXk - float(oS.labelMat[k])
    return Ek

# 选择内循环的alpha值
def selectJ(i, oS, Ei):
    maxK = -1
    maxDeltaE = 0
    Ej = 0
    oS.eCache[i] = [1, Ei]
    validecachelist = np.nonzero(oS.eCache[:, 0].A)[0]    # 返回非0的E值对应的alpha值
    if len(validecachelist) > 1:
        for k in validecachelist:
            if k == i:
                continue
            Ek = calcEk(oS, k)
            deltaE = abs(Ei - Ek)
            if deltaE > maxDeltaE:    # 选择具有最大步长的j
                maxK = k
                maxDeltaE = deltaE
                Ej = Ek
        return maxK, Ej
    else:
        j = selectJrand(i, oS.m)
        Ej = calcEk(oS, j)
    return j, Ej

def updateEk(oS, k):
    Ek = calcEk(oS, k)
    oS.eCache[k] = [1, Ek]

Platt SMO优化例程

In [4]:
def innerL(i, oS):
    Ei = calcEk(oS, i)
    if ((oS.labelMat[i] * Ei < -oS.tol) and (oS.alphas[i] < oS.C)) or ((oS.labelMat[i] * Ei > oS.tol) and (oS.alphas[i] >0)):
        j ,Ej = selectJ(i, oS, Ei)
        alphaIold = oS.alphas[i].copy()
        alphaJold = oS.alphas[j].copy()
        # 保证alpha在0与C之间
        if oS.labelMat[i] != oS.labelMat[j]:
            L = max(0, oS.alphas[j] - oS.alphas[i])
            H = min(oS.C, oS.C + oS.alphas[j] - oS.alphas[i])
        else:
            L = max(0, oS.alphas[j] - oS.alphas[i] - oS.C)
            H = min(oS.C, oS.alphas[j] - oS.alphas[i])
        if L == H:
            print('L == H')
            return 0
        # alpha[j]的最优修改量
        eta = 2.0 * oS.X[i, :] * oS.X[j, :].T - oS.X[i, :] * oS.X[i, :].T - oS.X[j, :] * oS.X[j, :].T
        if eta >= 0:
            print('eta >= 0')
            return 0
        # 计算新的alpha[j]
        oS.alphas[j] -= oS.labelMat[j] * (Ei - Ej) / eta
        oS.alphas[j] = clipAlpha(oS.alphas[j], H, L)
        updateEk(oS, i)
        # 设置常数项b
        b1 = oS.b - Ei - oS.labelMat[i] * (oS.alphas[i] - alphaIold) * oS.X[i, :] * oS.X[i].T - oS.labelMat[j] * (oS.alphas[j] - alphaJold) * oS.X[i, :] * oS.X[j, :].T
        b2 = oS.b - Ej - oS.labelMat[i] * (oS.alphas[i] - alphaIold) * oS.X[i, :] * oS.X[j].T - oS.labelMat[j] * (oS.alphas[j] - alphaJold) * oS.X[j, :] * oS.X[j, :].T
        if 0 < oS.alphas[i] and oS.C > oS.alphas[i]:
            oS.b = b1
        elif 0 < oS.alphas[j] and oS.C > oS.alphas[j]:
            oS.b = b2
        else:
            oS.b = (b1 + b2)/2.0
        return 1
    else:
        return 0

Platt SMO外循环代码

In [5]:
def smoP(dataMatIn, classLabels, C, toler, maxIter, kTup=('lin', 0)):
    oS = optStruct(np.mat(dataMatIn), np.mat(classLabels).transpose(), C, toler)
    iter = 0
    entireSet = True
    alphaPairsChanged = 0
    while iter < maxIter and (alphaPairsChanged > 0 or entireSet):
        alphaPairsChanged = 0
        if entireSet:    # 遍历所有值
            for i in range(oS.m):
                alphaPairsChanged += innerL(i,oS)
                print('fullSet, iter: %d i: %d, pairs changed %d' % (iter, i, alphaPairsChanged))
            iter += 1
        else:    # 遍历非边界值
            nonBoundIs = np.nonzero((oS.alphas.A > 0) * (oS.alphas.A < C))[0]
            for i in nonBoundIs:
                alphaPairsChanged += innerL(i, oS)
                print('non-bound, iter: %d i: %d, pairs changed %d' % (iter, i, alphaPairsChanged))
            iter += 1
        if entireSet:
            entireSet = False
        elif alphaPairsChanged == 0:
            entireSet = True
        print('iteration number: %d' % iter)
    return oS.b, oS.alphas

计算W

In [6]:
def calcWs(alphas, dataArr, classLabels):
    X = np.mat(dataArr)
    labelMat = np.mat(classLabels).transpose()
    m, n = np.shape(X)
    w = np.zeros((n, 1))
    for i in range(m):
        w += np.multiply(alphas[i] * labelMat[i], X[i, :].T)
    return w

In [7]:
dataArr, labelArr = loadDataSet('testSet.txt')
b, alphas = smoP(dataArr, labelArr, 0.6, 0.001, 40)

L == H
fullSet, iter: 0 i: 0, pairs changed 0
L == H
fullSet, iter: 0 i: 1, pairs changed 0
fullSet, iter: 0 i: 2, pairs changed 1
fullSet, iter: 0 i: 3, pairs changed 2
fullSet, iter: 0 i: 4, pairs changed 3
fullSet, iter: 0 i: 5, pairs changed 3
fullSet, iter: 0 i: 6, pairs changed 3
fullSet, iter: 0 i: 7, pairs changed 3
fullSet, iter: 0 i: 8, pairs changed 4
fullSet, iter: 0 i: 9, pairs changed 4
fullSet, iter: 0 i: 10, pairs changed 4
fullSet, iter: 0 i: 11, pairs changed 4
fullSet, iter: 0 i: 12, pairs changed 4
fullSet, iter: 0 i: 13, pairs changed 4
fullSet, iter: 0 i: 14, pairs changed 4
fullSet, iter: 0 i: 15, pairs changed 5
fullSet, iter: 0 i: 16, pairs changed 5
fullSet, iter: 0 i: 17, pairs changed 5
fullSet, iter: 0 i: 18, pairs changed 5
fullSet, iter: 0 i: 19, pairs changed 5
fullSet, iter: 0 i: 20, pairs changed 5
fullSet, iter: 0 i: 21, pairs changed 5
fullSet, iter: 0 i: 22, pairs changed 5
L == H
fullSet, iter: 0 i: 23, pairs changed 5
fullSet, iter: 0 i: 24, pairs

non-bound, iter: 27 i: 57, pairs changed 4
iteration number: 28
non-bound, iter: 28 i: 0, pairs changed 1
non-bound, iter: 28 i: 15, pairs changed 2
non-bound, iter: 28 i: 53, pairs changed 3
non-bound, iter: 28 i: 57, pairs changed 4
iteration number: 29
non-bound, iter: 29 i: 0, pairs changed 1
non-bound, iter: 29 i: 15, pairs changed 2
non-bound, iter: 29 i: 53, pairs changed 3
non-bound, iter: 29 i: 57, pairs changed 4
iteration number: 30
non-bound, iter: 30 i: 0, pairs changed 1
non-bound, iter: 30 i: 15, pairs changed 2
non-bound, iter: 30 i: 53, pairs changed 3
non-bound, iter: 30 i: 57, pairs changed 4
iteration number: 31
non-bound, iter: 31 i: 0, pairs changed 1
non-bound, iter: 31 i: 15, pairs changed 2
non-bound, iter: 31 i: 53, pairs changed 3
non-bound, iter: 31 i: 57, pairs changed 4
iteration number: 32
non-bound, iter: 32 i: 0, pairs changed 1
non-bound, iter: 32 i: 15, pairs changed 2
non-bound, iter: 32 i: 53, pairs changed 3
non-bound, iter: 32 i: 57, pairs changed

In [8]:
ws = calcWs(alphas, dataArr, labelArr)
ws

array([[0.70510008],
       [0.09333156]])

In [9]:
datMat = np.mat(dataArr)
datMat[0] * np.mat(ws) + b

matrix([[-0.20918498]])